In [ ]:
#!rm -r '/Users/aaron/neuefische/grain-boundary-detection/data/data_train/' 

Directories

In [ ]:
import os

def folder_structure(target_dir='../data/'):
    """Generates folder structure

    Args:
        target_dir (str, optional): The data directory path relative to the directory you run this function from. Defaults to '../data/'.
    """

    dir_lst = ['data_original', 'data_temp']
    subdir_lst = ['images/','masks/']
    parent_dir = os.getcwd()

    # make main directories
    for dir in dir_lst:
        for subdir in subdir_lst:
            os.makedirs(os.path.join(parent_dir, target_dir, dir, subdir), exist_ok=True)
        
    # # make subdirectories
    # for i in ['train','val','test']:
    #     for subdir in subdir_lst:
    #         os.makedirs(os.path.join(parent_dir, target_dir+'data_train', i+'_'+subdir+i), exist_ok=True)
    return

In [ ]:
folder_structure()

Create smaller images

In [ ]:
import cv2
from patchify import patchify
import numpy as np

In [ ]:
# load images and sort masks and images
train_img_dir = '../data/data_train/train/images/train/'
train_mask_dir = '../data/data_train/train/masks/train/'

img_list = os.listdir(train_img_dir)
img_list.sort()
msk_list = os.listdir(train_mask_dir)
msk_list.sort()
num_images = len(os.listdir(train_img_dir))
print("Total number of training images are: ", num_images)

# Plot of some images as a sanity check
img_num = random.randint(0, num_images-1)

img_for_plot = cv2.imread(train_img_dir+img_list[img_num], 0)
mask_for_plot =cv2.imread(train_mask_dir+msk_list[img_num], 0)

In [ ]:
def create_quarters(source_dir='../data/data_original/', target_dir='../data/data_original/'):
    """Creates smaller patches of `size` from large images in `source_dir` and copies them to subfolders in `target dir`.
    """
    # Create patches of images
    image_dir = source_dir+'images/'
    img_target_dir = target_dir+'images/'
    img_list = os.listdir(image_dir)
    img_list.sort()
    img_counter=1

    for file in img_list:
        if file.endswith('.tif'):
            print('chopping up ' + file)
            large_image = cv2.imread(image_dir+'/'+file, 0)
            width = large_image.shape[1]
            # Cut the image vertically into two halves
            width_cutoff = width // 2
            left1 = large_image[:, :width_cutoff]
            right1 = large_image[:, width_cutoff:]

            #Cut left halve into quarters
            large_image = cv2.rotate(left1, cv2.ROTATE_90_CLOCKWISE)
            width = large_image.shape[1]
            width_cutoff = width // 2
            l1 = large_image[:, :width_cutoff]
            l2 = large_image[:, width_cutoff:]
            #Save the left quarters
            l1 = cv2.rotate(l1, cv2.ROTATE_90_COUNTERCLOCKWISE)
            cv2.imwrite(img_target_dir + 'img_' + str(img_counter) + '_3' + '_' +'.tif', l1)
            l2 = cv2.rotate(l2, cv2.ROTATE_90_COUNTERCLOCKWISE)
            cv2.imwrite(img_target_dir + 'img_' + str(img_counter) + '_1' + '_' +'.tif', l2)

            #Cut right halve into quarters
            large_image = cv2.rotate(right1, cv2.ROTATE_90_CLOCKWISE)
            width = large_image.shape[1]
            width_cutoff = width // 2
            r1 = large_image[:, :width_cutoff]
            r2 = large_image[:, width_cutoff:]
            #Save the right quarters
            r1 = cv2.rotate(r1, cv2.ROTATE_90_COUNTERCLOCKWISE)
            cv2.imwrite(img_target_dir + 'img_' + str(img_counter) + '_4' + '_' +'.tif', r1)
            r2 = cv2.rotate(r2, cv2.ROTATE_90_COUNTERCLOCKWISE)
            cv2.imwrite(img_target_dir + 'img_' + str(img_counter) + '_2' + '_' +'.tif', r2)
            

            img_counter+=1

    # Create patches of masks
    mask_dir = source_dir+'masks/'
    msk_target_dir = target_dir+'masks/'
    img_list = os.listdir(mask_dir)
    img_list.sort()
    msk_counter=1
    
    for file in img_list:
        if file.endswith('.tif'):
            print('chopping up ' + file)
            large_image = cv2.imread(mask_dir+'/'+file, 0)
            width = large_image.shape[1]
            # Cut the image vertically into two halves
            width_cutoff = width // 2
            left1 = large_image[:, :width_cutoff]
            right1 = large_image[:, width_cutoff:]

            #Cut left halve into quarters
            large_image = cv2.rotate(left1, cv2.ROTATE_90_CLOCKWISE)
            width = large_image.shape[1]
            width_cutoff = width // 2
            l1 = large_image[:, :width_cutoff]
            l2 = large_image[:, width_cutoff:]
            #Save the left quarters
            l1 = cv2.rotate(l1, cv2.ROTATE_90_COUNTERCLOCKWISE)
            cv2.imwrite(msk_target_dir + 'msk_' + str(msk_counter) + '_3' + '_' +'.tif', l1)
            l2 = cv2.rotate(l2, cv2.ROTATE_90_COUNTERCLOCKWISE)
            cv2.imwrite(msk_target_dir + 'msk_' + str(msk_counter) + '_1' + '_' +'.tif', l2)

            #Cut right halve into quarters
            large_image = cv2.rotate(right1, cv2.ROTATE_90_CLOCKWISE)
            width = large_image.shape[1]
            width_cutoff = width // 2
            r1 = large_image[:, :width_cutoff]
            r2 = large_image[:, width_cutoff:]
            #Save the right quarters
            r1 = cv2.rotate(r1, cv2.ROTATE_90_COUNTERCLOCKWISE)
            cv2.imwrite(msk_target_dir + 'msk_' + str(msk_counter) + '_4' + '_' +'.tif', r1)
            r2 = cv2.rotate(r2, cv2.ROTATE_90_COUNTERCLOCKWISE)
            cv2.imwrite(msk_target_dir + 'msk_' + str(msk_counter) + '_2' + '_' +'.tif', r2)
            
            msk_counter+=1
    
    print('Done.')

In [ ]:
create_quarters()

In [ ]:
def create_tiles(size=512, step=512, source_dir='../data/data_original/', target_dir='../data/data_temp/', remove=True):
    """Creates smaller patches of `size` from large images in `source_dir` and copies them to subfolders in `target dir`.

    Args:
        size (int, optional): Size of the patches in pixels. Defaults to 512. Make sure the original image dimension are multiples of this size.
        source_dir (str, optional): Directory where the large images are stored. Defaults to '../data/data_original/'.
        target_dir (str, optional): Directory where the patched images will be stored. Defaults to '../data/data_temp/'.
        remove (boolean, optional): Whether to remove masks and their corresponding images that contain 1 or fewer features.
    """
    # Create patches of images
    image_dir = source_dir+'images/'
    img_target_dir = target_dir+'images/'
    img_list = os.listdir(image_dir)
    img_list.sort()
    img_counter=1

    for file in img_list:
        if file.endswith('.tif'):
            print('chopping up ' + file)
            large_image = cv2.imread(image_dir+'/'+file, 0)
            patches = patchify(large_image, (size, size), step=step)
            for i in range(patches.shape[0]):
                for j in range(patches.shape[1]):
                    single_patch_img = patches[i, j]
                    if not cv2.imwrite(img_target_dir + 'img_' + str(img_counter) + '_'+ str(i)+'_'+str(j)+'.tif', single_patch_img):
                        raise Exception("Could not write the image")
            img_counter+=1

    # Create patches of masks
    mask_dir = source_dir+'masks/'
    msk_target_dir = target_dir+'masks/'
    img_list = os.listdir(mask_dir)
    img_list.sort()
    msk_counter=1
    
    for file in img_list:
        if file.endswith('.tif'):
            print('chopping up ' + file)
            large_image = cv2.imread(mask_dir+'/'+file, 0)
            patches = patchify(large_image, (size, size), step=step)
            for i in range(patches.shape[0]):
                for j in range(patches.shape[1]):
                    single_patch_img = patches[i, j]
                    if not cv2.imwrite(msk_target_dir + 'msk_' + str(msk_counter) + '_'+ str(i)+'_'+str(j)+'.tif', single_patch_img):
                        raise Exception("Could not write the image")
            msk_counter+=1

    if remove:
        # Check if masks contain only one or fewer features and add them to a list
        unwanted=[]
        print('getting low feature masks...')
        for msk_patch in os.listdir(msk_target_dir):
            if len(np.unique(cv2.imread(msk_target_dir + msk_patch, 0))) <= 1:
                unwanted.append(msk_patch)
                os.remove(os.path.join(msk_target_dir + msk_patch))

        # Check which images correspond to those masks and delete them
        print(f'removing {len(unwanted)} mask- and corresponding image patches...')
        for img_patch in os.listdir(img_target_dir):
            for entry in unwanted:
                if img_patch[3:] == entry[3:]:
                    os.remove(os.path.join(img_target_dir+img_patch))
    
    print('Done.')
    return

In [ ]:
create_tiles()

Train/Val split

In [ ]:
import splitfolders
import shutil

def split_folders(input_dir='../data/data_temp/', output_dir='../data/data_train/', ratio=(0.8,0.2), seed=42):
    """Splits available data according to a given ratio and copies it to a specified folder.

    Args:
        input_dir (str, optional): Path to directory containing the image tiles. Defaults to '../data/data_temp'.
        output_dir (str, optional): Path to the training directory. Defaults to '..data/data_train'.
        ratio (tuple, optional): Train/validation/test split ratio. Defaults to (0.8,0.2).
        seed (int, optional): Random seed. Defaults to 42.
    """
    # Create training folder
    cwd_dir = os.getcwd()
    os.makedirs(os.path.join(cwd_dir, output_dir), exist_ok=True )

    # Split the data into train,val,test parts and move them to the training folder
    splitfolders.ratio(input_dir, output=output_dir, seed=seed, ratio=ratio, group_prefix=None, move=True)
    print(f'Done splitting with a train/val/test ratio of {ratio}')

    # This part moves the images to a subfolder because we are doing multiclass segmentation and keras' datagenerator flow_from_directory expects at least one folder
    dir_lst=['train/','val/','test/']
    dir_lst2=['images/','masks/']

    for dir in dir_lst:
        for dir2 in dir_lst2:
            os.makedirs(os.path.join(output_dir+dir+dir2+dir), exist_ok=True)
            for file_name in os.listdir(os.path.join(output_dir+dir+dir2)):
                # file paths
                source = output_dir + dir + dir2 + file_name
                destination = output_dir + dir + dir2 + dir + file_name
                # move bitch!
                if os.path.isfile(source):
                    shutil.move(source, destination)
    return


In [ ]:
split_folders()